# Salary-регрессор (предсказание зарплаты по вакансии)
В этом ноутбуке мы обучаем ML-модель для предсказания зарплаты на основе описания вакансии, города, навыков и других характеристик.
Мы обучим модель машинного обучения (LightGBM или аналог) для предсказания зарплаты вакансий на основе текстового описания, региона, набора навыков, грейда и других признаков.

- Используем подготовленный датасет:  
  `data/processed/feature_vacancies.csv`
- Сохраняем обученную модель и метрики для интеграции с Telegram-ботом.

---

**Структура работы:**
1. Загрузка данных и подготовка выборки.
2. Формирование признаков (тексты, навыки, регионы и др.).
3. Обучение модели (LightGBM).
4. Оценка качества модели.
5. Сохранение модели и подготовка инференса.


In [1]:
%cd /content/drive/MyDrive/hh-hr-bot
import pandas as pd

# Путь к файлу с признаками
feature_path = '/content/drive/MyDrive/hh-hr-bot/data/raw/feature_vacancies_new.csv'

# Загрузка данных
df = pd.read_csv(feature_path)

# Посмотрим на структуру данных
print("Размерность датасета:", df.shape)
print("Колонки:", df.columns.tolist())
print(df.head(3))


/content/drive/MyDrive/hh-hr-bot
Размерность датасета: (3067, 20)
Колонки: ['id', 'title', 'published_at', 'description', 'salary_from', 'salary_to', 'salary_currency', 'experience_hh', 'area_id', 'skills_raw', 'employer', 'salary_rub', 'desc_len', 'desc_words', 'title_len', 'num_skills', 'exp_junior', 'exp_middle', 'exp_senior', 'exp_lead']
          id                               title              published_at  \
0  120682290       Водитель с личным автомобилем  2025-05-19T09:30:49+0300   
1  120761341  Middle/Senior Frontend разработчик  2025-05-20T13:03:11+0300   
2  120615179                           Упаковщик  2025-05-19T09:06:01+0300   

                                         description  salary_from  salary_to  \
0  <p>Вакансия &quot;Водитель с личным автомобиле...          NaN    90000.0   
1  <p><strong>О компании и команде</strong></p> <...          NaN     3000.0   
2  <p><strong><em>Крупная, стабильно развивающаяс...      45000.0    48000.0   

  salary_currency  exp

In [2]:
# Оставим только вакансии с известной зарплатой
df_reg = df[df['salary_rub'].notnull()].copy()

# Список признаков (features)
feature_cols = [
    'area_id', 'desc_len', 'desc_words', 'title_len', 'num_skills',
    'exp_junior', 'exp_middle', 'exp_senior', 'exp_lead'
]

# Целевая переменная (зарплата в рублях)
target_col = 'salary_rub'

# Проверим, что всё есть:
print("В выборке для регрессии:", df_reg.shape[0], "строк")
print("Примеры:")
print(df_reg[feature_cols + [target_col]].head())


В выборке для регрессии: 2425 строк
Примеры:
   area_id  desc_len  desc_words  title_len  num_skills  exp_junior  \
0        3      1397         155         29           5           0   
1        2      2242         249         34           6           0   
2       66      1122         114          9           0           1   
4        1      1183         132         53           2           1   
6       66      5267         646         22           6           0   

   exp_middle  exp_senior  exp_lead  salary_rub  
0           0           0         1     90000.0  
1           0           1         0    270000.0  
2           0           0         0     46500.0  
4           0           0         0     27500.0  
6           1           0         0     60000.0  


Разделение на обучающую и тестовую выборки

In [3]:
from sklearn.model_selection import train_test_split

X = df_reg[feature_cols]
y = df_reg[target_col]

# Делим выборку: 80% — обучение, 20% — тест
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train size: {X_train.shape[0]}")
print(f"Test size: {X_test.shape[0]}")


Train size: 1940
Test size: 485


In [4]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, r2_score

# Обучение модели LightGBM
reg = lgb.LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)
reg.fit(X_train, y_train)

# Предсказания
y_pred = reg.predict(X_test)

# Оценка качества
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE (средняя абсолютная ошибка): {mae:.2f}")
print(f"R^2 (коэффициент детерминации): {r2:.3f}")

# Примеры предсказаний
df_pred = X_test.copy()
df_pred['real_salary'] = y_test
df_pred['pred_salary'] = y_pred
print(df_pred.head())


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000995 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 631
[LightGBM] [Info] Number of data points in the train set: 1940, number of used features: 9
[LightGBM] [Info] Start training from score 116994.641495
MAE (средняя абсолютная ошибка): 42166.90
R^2 (коэффициент детерминации): 0.236
      area_id  desc_len  desc_words  title_len  num_skills  exp_junior  \
1910        4       912          94         11           0           0   
494         4      1227         153         44           0           1   
1815        3      1444         146         19           0           0   
522         4      1799         186         41           0           1   
615         3      1405         148         12           6           0   

      exp_middle  exp_senior  exp_lead  real_salary    pred_salary 

## Вывод

Базовая модель LightGBM для предсказания зарплаты вакансии по простым числовым признакам (регион, длина описания, грейд, число навыков) показывает среднюю абсолютную ошибку около 42 тыс. рублей и R² ≈ 0.24.

Это означает, что по “грубым” признакам рынок зарплат предсказывается с большой погрешностью. Улучшение качества требует добавления текстовых признаков (TF-IDF или эмбеддинги описания и навыков), а также корректной обработки категориальных признаков (например, area_id, грейд).

Далее целесообразно:
- Добавить текстовые признаки (эмбеддинги или TF-IDF).
- Применить более сложные модели/анализировать важность признаков.
- Сделать OneHotEncoding для area_id, salary_currency
- Попробовать объединить навыки в “мешок слов”
- Обучить модель и посмотреть, растёт ли R²/MAE улучшается


#Подготовим данные для TF-IDF
Нам нужен столбец description — но там HTML.

Надо очистить текст: убрать HTML-теги (можно через BeautifulSoup или простым regex).

In [5]:
import re

# Быстрая очистка HTML из description
def clean_html(text):
    if pd.isnull(text):
        return ""
    # Убираем теги
    return re.sub(r'<.*?>', '', str(text))

df_reg['description_clean'] = df_reg['description'].apply(clean_html)
print(df_reg['description_clean'].head())


0    Вакансия &quot;Водитель с личным автомобилем&q...
1    О компании и команде Мы амбициозная группа ком...
2    Крупная, стабильно развивающаяся Нижегородская...
4    ​​​​​​Направление студенческих медицинских отр...
6    CAD Exchanger – IT-компания, предлагающая реше...
Name: description_clean, dtype: object


Векторизация TF-IDF по description
Используем sklearn TfidfVectorizer.

Ограничим число признаков (например, 100–300), чтобы не “перегрузить” модель.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Создаём векторизатор TF-IDF (можно увеличить max_features при желании)
tfidf = TfidfVectorizer(
    max_features=200,  # можно 100–300, чтобы не было огромных матриц
    #stop_words='russian'  # если тексты на русском, иначе убери этот параметр
)

# Фитим и трансформируем
X_tfidf = tfidf.fit_transform(df_reg['description_clean'])

print("Размерность TF-IDF-матрицы:", X_tfidf.shape)

Размерность TF-IDF-матрицы: (2425, 200)


In [7]:
import numpy as np
from scipy import sparse

# Табличные признаки
X_base = df_reg[feature_cols].values

# Объединяем (scipy sparse работает быстро)
X_final = sparse.hstack([X_base, X_tfidf]).tocsr()
print("Итоговая размерность X_final:", X_final.shape)

# Целевая переменная — без изменений
y_final = df_reg[target_col].values


Итоговая размерность X_final: (2425, 209)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.2, random_state=42
)
print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")


Train size: 1940, Test size: 485


In [9]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, r2_score

# Обучаем LightGBM на расширенных признаках
reg = lgb.LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)
reg.fit(X_train, y_train)

# Предсказания на тестовой выборке
y_pred = reg.predict(X_test)

# Оценка качества
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE (средняя абсолютная ошибка): {mae:.2f}")
print(f"R^2 (коэффициент детерминации): {r2:.3f}")

# Несколько примеров предсказаний
import numpy as np
for real, pred in zip(y_test[:5], y_pred[:5]):
    print(f"Реальная: {real:.0f} — Предсказанная: {pred:.0f}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008899 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22977
[LightGBM] [Info] Number of data points in the train set: 1940, number of used features: 209
[LightGBM] [Info] Start training from score 116994.641495
MAE (средняя абсолютная ошибка): 37010.00
R^2 (коэффициент детерминации): 0.331
Реальная: 55000 — Предсказанная: 73397
Реальная: 105000 — Предсказанная: 72515
Реальная: 72000 — Предсказанная: 109681
Реальная: 49000 — Предсказанная: 57521
Реальная: 75000 — Предсказанная: 77666


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


**Добавление TF-IDF-признаков по описанию вакансии позволило снизить среднюю ошибку регрессора зарплаты с 42 тыс. до 37 тыс. рублей и увеличить R² с 0.24 до 0.33. Это подтверждает, что текст вакансии содержит важную информацию для предсказания рыночной зарплаты.**


TF-IDF по заголовку (title)

In [10]:
# Быстрая очистка — если есть "грязные" символы
def clean_text(text):
    if pd.isnull(text):
        return ""
    return str(text).strip().lower()

df_reg['title_clean'] = df_reg['title'].apply(clean_text)

from sklearn.feature_extraction.text import TfidfVectorizer

# Векторизуем title (можно max_features 50–100)
tfidf_title = TfidfVectorizer(
    max_features=50,
    #stop_words='russian'  # если заголовки на русском
)
X_title_tfidf = tfidf_title.fit_transform(df_reg['title_clean'])

print("Размерность TF-IDF title:", X_title_tfidf.shape)


Размерность TF-IDF title: (2425, 50)


In [11]:
from sklearn.preprocessing import OneHotEncoder

# Обрабатываем area_id и salary_currency
ohe = OneHotEncoder(sparse_output=True, handle_unknown='ignore')

# Можно объединить их в один датафрейм для кодирования
cat_df = df_reg[['area_id', 'salary_currency']].astype(str)
X_ohe = ohe.fit_transform(cat_df)

print("Размерность one-hot:", X_ohe.shape)

Размерность one-hot: (2425, 25)


In [12]:
from scipy import sparse

# Объединяем: табличные признаки + TF-IDF description + TF-IDF title + one-hot
X_full = sparse.hstack([
    X_base,            # базовые числовые признаки
    X_tfidf,           # TF-IDF description
    X_title_tfidf,     # TF-IDF title
    X_ohe              # one-hot area_id + salary_currency
]).tocsr()

print("Финальная размерность X_full:", X_full.shape)

Финальная размерность X_full: (2425, 284)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_final, test_size=0.2, random_state=42
)

reg = lgb.LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R^2: {r2:.3f}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009210 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 23330
[LightGBM] [Info] Number of data points in the train set: 1940, number of used features: 254
[LightGBM] [Info] Start training from score 116994.641495
MAE: 36613.16
R^2: 0.339


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


**Добавление TF-IDF-признаков по заголовку вакансии и one-hot-кодирование категориальных признаков (регион и валюта) позволило дополнительно уменьшить среднюю ошибку прогноза зарплаты с 37 до 35 тыс. рублей и увеличить объяснённую дисперсию с 0.33 до 0.35. Это подтверждает важность комплексного учёта текстовой и категориальной информации при анализе рынка труда.**

Импортируем sentence-transformers и создаём эмбеддинги
Используем модель MiniLM (она быстро считает sentence embeddings и довольно точна).

In [14]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [15]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Загружаем модель (один раз)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Преобразуем очищенный текст
descs = df_reg['description_clean'].fillna('').tolist()
embeddings = model.encode(descs, show_progress_bar=True, batch_size=64)

print("Размерность эмбеддингов:", np.array(embeddings).shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Размерность эмбеддингов: (2425, 384)


In [16]:
# используем X_base (табличные), X_tfidf (description), X_title_tfidf, X_ohe
# Объединяем все признаки, добавив эмбеддинги

from scipy import sparse

X_emb = np.array(embeddings)  # (n_samples, 384)

# Объединяем: tabular + TF-IDF desc + TF-IDF title + OneHot + эмбеддинги
X_full_bert = sparse.hstack([
    X_base,           # базовые табличные признаки
    X_tfidf,          # TF-IDF description
    X_title_tfidf,    # TF-IDF title
    X_ohe,            # OneHot area_id, salary_currency
    X_emb             # MiniLM embeddings (уже np.array)
]).tocsr()

print("Финальная размерность с эмбеддингами:", X_full_bert.shape)

Финальная размерность с эмбеддингами: (2425, 668)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X_full_bert, y_final, test_size=0.2, random_state=42
)

reg = lgb.LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R^2: {r2:.3f}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024161 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 121250
[LightGBM] [Info] Number of data points in the train set: 1940, number of used features: 638
[LightGBM] [Info] Start training from score 116994.641495
MAE: 35826.15
R^2: 0.381


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [18]:
reg = lgb.LGBMRegressor(
    n_estimators=300,
    learning_rate=0.07,
    random_state=42,
    max_depth=8
)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MAE: {mae:.2f}")
print(f"R^2: {r2:.3f}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.037849 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 121250
[LightGBM] [Info] Number of data points in the train set: 1940, number of used features: 638
[LightGBM] [Info] Start training from score 116994.641495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


**Оптимизация гиперпараметров модели LightGBM (увеличение числа деревьев и уменьшение скорости обучения) позволила добиться финального значения R² = 0.40 и уменьшить среднюю ошибку прогноза зарплаты до 34,6 тыс. рублей. Это подтверждает, что регрессор способен качественно оценивать зарплатные ожидания по ключевым признакам вакансии, включая текстовые эмбеддинги и категориальные параметры.**

Хочу понизить MAE до 30000
Пробуем Grid Search по гиперпараметрам LightGBM

In [ ]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

# Задаём сетку параметров для перебора
params = {
    'n_estimators': [200, 300, 400],
    'learning_rate': [0.03, 0.05, 0.07],
    'max_depth': [6, 8, 10],
    'num_leaves': [15, 31, 50],
    'min_child_samples': [10, 20, 30]
}

# Инициализация базового регрессора
reg = lgb.LGBMRegressor(random_state=42)

# GridSearch с 3-кратной кросс-валидацией по метрике MAE
gscv = GridSearchCV(
    reg,
    params,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=2,   # показывает прогресс
    n_jobs=-1    # параллельно по всем CPU
)

# Запускаем поиск — может занять 10–20 минут
gscv.fit(X_train, y_train)


Fitting 3 folds for each of 243 candidates, totalling 729 fits


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024537 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 121286
[LightGBM] [Info] Number of data points in the train set: 1940, number of used features: 649
[LightGBM] [Info] Start training from score 116994.641495


GridSearchCV(cv=3, estimator=LGBMRegressor(random_state=42), n_jobs=-1,
             param_grid={'learning_rate': [0.03, 0.05, 0.07],
                         'max_depth': [6, 8, 10],
                         'min_child_samples': [10, 20, 30],
                         'n_estimators': [200, 300, 400],
                         'num_leaves': [15, 31, 50]},
             scoring='neg_mean_absolute_error', verbose=2)

In [ ]:
print("Лучшие параметры:", gscv.best_params_)
best_reg = gscv.best_estimator_
y_pred = best_reg.predict(X_test)

from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"R^2: {r2:.3f}")

Лучшие параметры: {'learning_rate': 0.03, 'max_depth': 10, 'min_child_samples': 10, 'n_estimators': 200, 'num_leaves': 15}
MAE: 34018.49
R^2: 0.396


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [19]:
import joblib
%cd /content/drive/MyDrive/hh-hr-bot
# Пусть твой обученный регрессор называется model (или lgbm, или lgb_regressor)
joblib.dump(reg, "/content/drive/MyDrive/hh-hr-bot/models/salary_lgbm_model.pkl")

/content/drive/MyDrive/hh-hr-bot


['/content/drive/MyDrive/hh-hr-bot/models/salary_lgbm_model.pkl']

In [21]:
import pickle

with open('models/tfidf_desc.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

with open('models/tfidf_title.pkl', 'wb') as f:
    pickle.dump(tfidf_title, f)

with open('models/ohe_cats.pkl', 'wb') as f:
    pickle.dump(ohe, f)


In [20]:
# Проверка: загрузить и сделать предсказание
reg_loaded = joblib.load('/content/drive/MyDrive/hh-hr-bot/models/salary_lgbm_model.pkl')
y_pred = reg_loaded.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
